In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
# Database Reading and Manipulation
import pandas as pd

# Linear Algebra
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Processing results
import json

# Model Selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV

# Modeling
from sklearn.linear_model import LinearRegression

# Processing
from sklearn.preprocessing import StandardScaler

# Data imputation
from sklearn.impute import SimpleImputer

# Pipeline
from sklearn.pipeline import Pipeline

# Metrics
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score


# Custom modules
## Model selection
from src.cross_validation.blocking_time_series_split import BlockingTimeSeriesSplit

## Function to print scores
from src.utils.print_scores import print_scores

## Function to calculate score regression metrics
from src.utils.score_regression_metrics import score_regression_metrics

## Function to fill the results metric dict
from src.utils.fill_results_dict import fill_results_dict

<IPython.core.display.Javascript object>

# Functions and definitions

In [3]:
SEED = 47
METRICS = (
    "neg_root_mean_squared_error",
    "neg_mean_absolute_error",
    "neg_mean_absolute_percentage_error",
    "r2",
)
METRICS_DICT = {
    "neg_root_mean_squared_error": "RMSE",
    "neg_mean_absolute_error": "MAE",
    "neg_mean_absolute_percentage_error": "MAPE",
    "r2": "R2",
}

<IPython.core.display.Javascript object>

## Defining a dataframe structure to save the results

In [4]:
results_to_save = []

results_dict = {
    "Category": "Local Model",
    "Company": "partner_ii",
    "Features": "Chemical + Mineralogical + Physical",
    "Data Shape": None,
    "Timesteps": None,
    "Model": "Linear Regression",
    "Model Params": None,
    "Scaler": "Standard Scaler",
    "Scaler Params": None,
    "Imputer": "Median",
    "Imputer Params": None,
    "Cross Validation": None,
    "Cross Validation Params": np.nan,
    "RMSE Train": np.nan,
    "MAE Train": np.nan,
    "MAPE Train": np.nan,
    "R2 Train": np.nan,
    "RMSE Test": np.nan,
    "MAE Test": np.nan,
    "MAPE Test": np.nan,
    "R2 Test": np.nan,
}

<IPython.core.display.Javascript object>

# Reading the dataset

In [5]:
df = pd.read_csv("../../../../../data/processed/partner_ii/Type-III.csv")

<IPython.core.display.Javascript object>

## Defining Features

In this set of experiments we keep only chemical and mineralogical features yielded by the same testing method/procedure

In [6]:
df_copy = df.copy()

<IPython.core.display.Javascript object>

# 1. Linear Regression

<h2>1. Dataset: df_no_cs</h2> <br>In this dataset the CS1, CS3  and CS7 variables are not considered. Only Chemical and mineralogical features measured by the same method. For this particular dataset, all chemical features, with the exception of LOI were measured by XRF and XRD methods.

In [7]:
y = df_copy.pop("CS28").values
x = df_copy.drop(["Date"], axis=1)
dates = df["Date"].copy()

<IPython.core.display.Javascript object>

## 1.1 Repeated KFold Cross validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Repeats:</b>10<br>
<b>Splits:</b>10<br>
    1. 10 folds of 123 samples each
    2. 90% train (1111 samples each fold)
    3. 10% test (123 samples each fold)
<b>Total:</b> 100 models<br>

In [8]:
repeats = 3
n_splits = 5
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = RepeatedKFold(n_splits=n_splits, n_repeats=repeats, random_state=SEED)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Repeated Cross Validation:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Repeated KFold"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 3}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Repeated Cross Validation:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.350 (0.050)
MAE: -1.058 (0.039)
MAPE: -0.022 (0.001)
R2: 0.741 (0.015)


******
[TEST]
******
RMSE: -1.593 (0.362)
MAE: -1.227 (0.205)
MAPE: -0.026 (0.004)
R2: 0.615 (0.196)




<IPython.core.display.Javascript object>

In [9]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model Cross Validation  \
                                                                             
0  Chemical + Mineralogical + Physical  Linear Regression   Repeated KFold   

  RMSE Test            MAE Test           MAPE Test             R2 Test  \
       mean       std      mean       std      mean       std      mean   
0  1.593318  0.362399  1.227425  0.204851  0.025988  0.004458  0.614566   

             
        std  
0  0.195716

<IPython.core.display.Javascript object>

## 1.2. Blocking Time Series Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>5<br>    
    1. 5 folds of 246 samples
    2. 50% train (123 samples each fold)
    3. 50% test (123 samples each fold)
<b>Total:</b> 5 models<br>

In [10]:
n_splits = 5
train_size = 0.8

pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = BlockingTimeSeriesSplit(n_splits=n_splits, train_size=train_size)
scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Blocking Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Blocking Time Series Split"
results_dict_copy[
    "Cross Validation Params"
] = '{"N_Splits": 5, "Repeats": 1, "train_size": 0.8}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Blocking Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -0.921 (0.120)
MAE: -0.731 (0.094)
MAPE: -0.015 (0.002)
R2: 0.801 (0.034)


******
[TEST]
******
RMSE: -3.361 (1.937)
MAE: -2.393 (0.894)
MAPE: -0.052 (0.019)
R2: -3.148 (4.182)




<IPython.core.display.Javascript object>

In [11]:
pd.concat(results_to_save).reset_index().groupby(
    ["Features", "Model", "Cross Validation"], dropna=False
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                            
0  Chemical + Mineralogical + Physical  Linear Regression   
1  Chemical + Mineralogical + Physical  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  3.361335  1.936532  2.393390  0.894072   
1              Repeated KFold  1.593318  0.362399  1.227425  0.204851   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.051604  0.018765 -3.148399  4.182299  
1  0.025988  0.004458  0.614566  0.195716

<IPython.core.display.Javascript object>

## 1.3. Time Series Split Cross Validation

The training set has size i * n_samples // (n_splits + 1) + n_samples % (n_splits + 1) in the i th split, with a test set of size n_samples//(n_splits + 1) by default, where n_samples is the number of samples.


<b>Dataset shape:</b> (1234, 38)<br>
<b>Splits:</b>10<br>    
    1. Train: 10 folds of 114, 226, 338, 450, 562, 675, 787, 899, 1011, 1123 samples each fold
    2. Test: 112 samples each fold
<b>Total:</b> 10 models<br>

In [12]:
n_splits = 5
gap = 0
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)
cv = TimeSeriesSplit(gap=gap, max_train_size=None, n_splits=n_splits, test_size=None)

scores = cross_validate(
    pipeline,
    x,
    y,
    scoring=METRICS,
    cv=cv,
    n_jobs=-1,
    return_train_score=True,
)
print("Time Series Split:")
print(f"Repeats: {repeats}")
print(f"n_splits: {n_splits}")
print()
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Time Series Split"
results_dict_copy["Cross Validation Params"] = '{"N_Splits": 5, "Repeats": 1, "Gap": 0}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(results_dict_copy, scores)
results_to_save.append(df_results)

Time Series Split:
Repeats: 3
n_splits: 5

******
[TRAIN]
******
RMSE: -1.205 (0.114)
MAE: -0.957 (0.090)
MAPE: -0.020 (0.002)
R2: 0.637 (0.038)


******
[TEST]
******
RMSE: -2.410 (1.118)
MAE: -1.855 (0.959)
MAPE: -0.039 (0.020)
R2: -0.982 (2.628)




<IPython.core.display.Javascript object>

In [13]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                            
0  Chemical + Mineralogical + Physical  Linear Regression   
1  Chemical + Mineralogical + Physical  Linear Regression   
2  Chemical + Mineralogical + Physical  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  3.361335  1.936532  2.393390  0.894072   
1              Repeated KFold  1.593318  0.362399  1.227425  0.204851   
2           Time Series Split  2.409888  1.117562  1.854684  0.959431   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.051604  0.018765 -3.148399  4.182299  
1  0.025988  0.004458  0.614566  0.195716  
2  0.039255  0.019804 -0.981661  2.627599

<IPython.core.display.Javascript object>

## 1.4. Out of time Split Cross Validation

<b>Dataset shape:</b> (1234, 38)<br>
<b>Train size: 80%</b><br>
<b>Test  size: 20%</b>


<b>Splits:</b> 2<br>    
    1. Train: 987
    2. Test: 247
<b>Total:</b> 1 model<br>

In [14]:
test_size = 0.2

x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=test_size, random_state=SEED, shuffle=False
)
pipeline = Pipeline(
    [
        ("imputer", SimpleImputer(strategy="median")),
        ("transformer", StandardScaler()),
        ("estimator", LinearRegression()),
    ]
)

pipeline.fit(x_train, y_train)

y_train_pred = pipeline.predict(x_train)
y_test_pred = pipeline.predict(x_test)

scores = score_regression_metrics(y_train, y_train_pred, y_test, y_test_pred)
print_scores(scores, METRICS, METRICS_DICT)

results_dict_copy = results_dict.copy()
results_dict_copy["Cross Validation"] = "Out of time Split"
results_dict_copy["Cross Validation Params"] = '{"Test Size": 0.2}'
results_dict_copy["Data Shape"] = x.shape
df_results = fill_results_dict(
    results_dict_copy, {key: [value] for key, value in scores.items()}
)
results_to_save.append(df_results)

******
[TRAIN]
******
RMSE: 1.300 (0.000)
MAE: 1.039 (0.000)
MAPE: 0.021 (0.000)
R2: 0.644 (0.000)


******
[TEST]
******
RMSE: 1.870 (0.000)
MAE: 1.439 (0.000)
MAPE: 0.033 (0.000)
R2: 0.473 (0.000)




<IPython.core.display.Javascript object>

In [15]:
pd.concat(results_to_save).groupby(["Features", "Model", "Cross Validation"])[
    ["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]
].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                            
0  Chemical + Mineralogical + Physical  Linear Regression   
1  Chemical + Mineralogical + Physical  Linear Regression   
2  Chemical + Mineralogical + Physical  Linear Regression   
3  Chemical + Mineralogical + Physical  Linear Regression   

             Cross Validation RMSE Test            MAE Test            \
                                   mean       std      mean       std   
0  Blocking Time Series Split  3.361335  1.936532  2.393390  0.894072   
1           Out of time Split  1.869793  0.000000  1.438560  0.000000   
2              Repeated KFold  1.593318  0.362399  1.227425  0.204851   
3           Time Series Split  2.409888  1.117562  1.854684  0.959431   

  MAPE Test             R2 Test            
       mean       std      mean       std  
0  0.051604  0.018765 -3.148399  4.182299  
1  0.033217  0.000000  0.473474  0.000000  
2  0.025988  0.004458  0.614566  0.195716  
3  0.039255  0.019804 -0.981661  2.627599

<IPython.core.display.Javascript object>

In [16]:
pipeline.named_steps["estimator"].coef_

array([ 0.19788274, -0.05525573, -0.12698662, -0.03703666, -0.01768227,
       -0.12292241,  0.03196855,  0.16838655,  0.10679496,  0.74292034,
        0.20200482,  0.12543686, -0.12218879,  0.10132247, -0.09516224,
       -0.08654083,  0.6461613 ,  0.70889452,  0.77354202, -0.14953877,
        0.06776243,  0.14966953])

<IPython.core.display.Javascript object>

In [17]:
pipeline.named_steps["estimator"].intercept_

48.225487179487175

<IPython.core.display.Javascript object>

In [18]:
coeff = pipeline.named_steps["estimator"].coef_
np.array(x.columns)[coeff == 0]

array([], dtype=object)

<IPython.core.display.Javascript object>

In [19]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns

Index(['cao', 'sio2', 'al2o3', 'fe2o3', 'mgo', 'so3', 'k2o', 'na2o', 'alite',
       'belite', 'aluminate', 'ferrite', 'free_lime', 'loi', 'water_demand',
       'setting_initial', 'CS1', 'CS3', 'CS7', 'blaine', 'sieve_32um',
       'sieve_45um'],
      dtype='object')

<IPython.core.display.Javascript object>

In [20]:
x.drop(np.array(x.columns)[coeff == 0], axis=1).columns.shape

(22,)

<IPython.core.display.Javascript object>

In [21]:
x.columns.shape

(22,)

<IPython.core.display.Javascript object>

In [22]:
coeffs = pd.DataFrame(
    {col: [c] for col, c in zip(x.columns, coeff)}, index=["Coefficients"]
)

<IPython.core.display.Javascript object>

In [23]:
coeffs.T["Coefficients"].sort_values(ascending=False).to_frame(
    name="Coefficients"
).style.background_gradient(axis=None, vmin=1, vmax=5, cmap="Greens")

,Coefficients
CS7,0.773542
belite,0.742920
CS3,0.708895
CS1,0.646161
aluminate,0.202005
cao,0.197883
na2o,0.168387
sieve_45um,0.149670
ferrite,0.125437
alite,0.106795


<IPython.core.display.Javascript object>

In [24]:
pd.concat(results_to_save).groupby(
    ["Features", "Model", "Cross Validation", "Cross Validation Params"]
)[["RMSE Test", "MAE Test", "MAPE Test", "R2 Test"]].agg(
    ["mean", lambda series: pd.Series(series.std(ddof=0), name="std")]
).reset_index().rename(
    columns={"<lambda_0>": "std"}
)

Features              Model  \
                                                            
0  Chemical + Mineralogical + Physical  Linear Regression   
1  Chemical + Mineralogical + Physical  Linear Regression   
2  Chemical + Mineralogical + Physical  Linear Regression   
3  Chemical + Mineralogical + Physical  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Test            \
                                                         mean       std   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}  3.361335  1.936532   
1                                {"Test Size": 0.2}  1.869793  0.000000   
2                     {"N_Splits": 5, "Repeats": 3}  1.593318  0.362399   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}  2.409888  1.117562   

   MAE Test           MAPE Test             R2 Test            
       mean       std      mean       std      mean       std  
0  2.393390  0.894072  0.051604  0.018765 -3.148399  4.182299  
1  1.438560  0.000000  0.033217  0.000000  0.473474  0.000000  
2  1.227425  0.204851  0.025988  0.004458  0.614566  0.195716  
3  1.854684  0.959431  0.039255  0.019804 -0.981661  2.627599

<IPython.core.display.Javascript object>

# Saving the results Dataframe

## Saving the full dataframe

In [25]:
path = "../../../../../reports/results/local_models/partner_ii/type-III/full/"
filename = "linear_regression_results_full_1.csv"

pd.concat(results_to_save).to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

## Saving the grouped dataframe

In [26]:
cols_groupby = [
    "Category",
    "Company",
    "Data Shape",
    "Timesteps",
    "Features",
    "Model",
    "Cross Validation",
    "Cross Validation Params",
]

cols_agg = ["RMSE Train", "MAE Train", "MAPE Train", "R2 Train"] + [
    "RMSE Test",
    "MAE Test",
    "MAPE Test",
    "R2 Test",
]

path = "../../../../../reports/results/local_models/partner_ii/type-III/grouped/"
filename = "linear_regression_results_grouped_1.csv"


df_results_to_save = (
    pd.concat(results_to_save)
    .groupby(cols_groupby, dropna=False)[cols_agg]
    .agg(["mean", lambda series: pd.Series(series.std(ddof=0), name="std")])
    .reset_index()
    .rename(columns={"<lambda_0>": "std"})
)

df_results_to_save.to_csv(
    path_or_buf=path + filename,
    mode="w",
    index=False,
    header=True,
)

<IPython.core.display.Javascript object>

In [27]:
df_results_to_save

Category     Company Data Shape Timesteps  \
                                                  
0  Local Model  partner_ii  (244, 22)       NaN   
1  Local Model  partner_ii  (244, 22)       NaN   
2  Local Model  partner_ii  (244, 22)       NaN   
3  Local Model  partner_ii  (244, 22)       NaN   

                              Features              Model  \
                                                            
0  Chemical + Mineralogical + Physical  Linear Regression   
1  Chemical + Mineralogical + Physical  Linear Regression   
2  Chemical + Mineralogical + Physical  Linear Regression   
3  Chemical + Mineralogical + Physical  Linear Regression   

             Cross Validation  \
                                
0  Blocking Time Series Split   
1           Out of time Split   
2              Repeated KFold   
3           Time Series Split   

                            Cross Validation Params RMSE Train            ...  \
                                                          mean       std  ...   
0  {"N_Splits": 5, "Repeats": 1, "train_size": 0.8}   0.920828  0.120205  ...   
1                                {"Test Size": 0.2}   1.300114  0.000000  ...   
2                     {"N_Splits": 5, "Repeats": 3}   1.349775  0.049558  ...   
3           {"N_Splits": 5, "Repeats": 1, "Gap": 0}   1.205231  0.114093  ...   

   R2 Train           RMSE Test            MAE Test           MAPE Test  \
       mean       std      mean       std      mean       std      mean   
0  0.801349  0.033903  3.361335  1.936532  2.393390  0.894072  0.051604   
1  0.644171  0.000000  1.869793  0.000000  1.438560  0.000000  0.033217   
2  0.741348  0.015451  1.593318  0.362399  1.227425  0.204851  0.025988   
3  0.637398  0.038210  2.409888  1.117562  1.854684  0.959431  0.039255   

              R2 Test            
        std      mean       std  
0  0.018765 -3.148399  4.182299  
1  0.000000  0.473474  0.000000  
2  0.004458  0.614566  0.195716  
3  0.019804 -0.981661  2.627599  

[4 rows x 24 columns]

<IPython.core.display.Javascript object>